In [8]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from collections import namedtuple
from statsmodels.stats.libqsturng import psturng

%matplotlib notebook

In [9]:
pcr = pd.read_csv('rtpcr_master.csv')
pcr = pcr.rename(
    columns={
        'Target': 'targ',
        'Sample': 'sample',
        'Spheroid Day': 'day',
        'Expression Mean': 'mean',
        'Expression SD': 'sd'
    })
pcr.describe()

,day,mean,sd
count,96.000000,96.000000,96.000000
mean,2.750000,1.392121,0.550040
std,2.695025,3.550555,1.995821
min,0.000000,0.000000,0.000000
25%,0.750000,0.000000,0.000000
50%,2.000000,0.443600,0.055385
75%,4.000000,1.112440,0.292637
max,7.000000,29.241570,15.811180


# Manual ANOVA Calculation
* 3 data points per row, 5 rows per day. n = 15 datapoints, df=14
    * Since each row has the same number of data points, we can calculate the grand mean from the group means.
* m = 5 groups, df=4
* n-m=10 error df
* SS between = sum of squares between group means and grand means
    * SS error sum of squares betweeen 


In [10]:
def manual_anova(means, sds, n):
    '''
    Performs a manual ANOVA from summary stats. n is the samples per mean.
    n must be the same for each group. 
    Can be modified later to support different n per group
    '''

    k = len(means)  # number of groups
    N = n * k
    grand_mean = np.mean(means)

    dfb = k - 1
    dfw = N - k
    dft = N - 1

    ssb = np.sum(n * (means - grand_mean)**2)
    ssw = np.sum((n - 1) * (sds**2))
    sst = ssb + ssw

    msb = ssb / dfb
    msw = ssw / dfw

    f = msb / msw
    p = 1 - stats.f.cdf(f, dfb, dfw)

    Stats = namedtuple(
        'Stats',
        'means, sds, n, k, N, grand_mean, dfb, dfw, dft, ssb, ssw, sst, msb, msw, f, p'
    )
    return Stats(means, sds, n, k, N, grand_mean, dfb, dfw, dft, ssb, ssw, sst,
                 msb, msw, f, p)

In [11]:
def manual_tukey(S, p_cutoff=0.05):
    '''
    Calculates Tukey's HSD. S is a namedtuple created by manual_anova()
    '''
    p, hsd, sig  = (np.zeros([S.k, S.k]) for _ in range(3))
    sig[:] = False
    hsd[:] = np.nan    
    p[:] = np.nan
    psig = np.array(p, dtype=str)
    psig[:] = 'NaN'

    means = S.means.to_numpy()
    sig_list = []

    denom = np.sqrt(S.msw / S.n)

    for i in range(0, S.k - 1):
        for j in range(i + 1, S.k):

            mi = means[i]
            mj = means[j]

            hsd[i][j] = np.abs(mi - mj) / denom
            p[i][j] = psturng(hsd[i][j], S.k, S.dfw)
            psig[i][j] = f'{p[i][j]:n}'

            if p[i][j] < p_cutoff:
                sig[i][j] = True
                sig_list.append((i, j))
                psig[j][i] = 'Sig'
            else:
                psig[j][i] = 'Not Sig'
    

    Stats = namedtuple('Stats', 'p, hsd, sig, sig_list, psig')
    return Stats(p, hsd, sig, sig_list, psig)

In [12]:
def print_tukey(arr, names, title='Tukey', min_lim= 8, max_lim=16):
    '''
    Pretty prints the results of manual_tukey()
    arr is an array of shape (len(names),len(names))
    min_lim and max_lim is the range of column sizes you want to allow
    '''
    names = [s[:max_lim] for s in names]
    lengths = [len(s) for s in names]
    max_len = max(lengths)
    print(f"{title:^{max_len}}", end='')
    [print(f' {names[i]}', end='') for i in range(len(names))]
    print('')

    for i in range(len(names)):
        print(f'{names[i]:>{max_len}}', end='')

        for j in range(len(names)):
            print(f' {arr[i][j]:>{lengths[j]}}', end='')

        print('')    

In [13]:
targets = np.unique(pcr['targ'])
n = 3
anovas, tukeys, samples = {}, {}, {}

for t in targets:
    
    tdf = pcr[pcr['targ'] == t]
    days = np.unique(tdf['day'])
    
    for d in days:
        
        day_df = tdf[tdf['day'] == d]
        means = day_df['mean']
        sds = day_df['sd']
        t_d = f'{t}_{d}'
        
        anovas[t_d] = manual_anova(means, sds, n)
        tukeys[t_d] = manual_tukey(anovas[t_d])
        samples[t_d] = day_df['sample']

In [16]:
from contextlib import redirect_stdout

with open('anova_results.txt', 'w') as f:
    with redirect_stdout(f):
        for k, v in anovas.items():
            if anovas[k].p < 0.05:
                print(f'{k} Overall | Significant | p: {anovas[k].p:n} | F: {anovas[k].f:n}')
            else:
                print(f'{k} Overall | Not Significant | p: {anovas[k].p:n} | F: {anovas[k].f:n}')
            samp_names = samples[k].to_numpy()
            print_tukey(tukeys[k].psig, samp_names, k)
            print('')

In [15]:
# for k,v in S._asdict().items():
#     print(f'{k}: {v}')

# Test manual tukey
T = manual_tukey(anovas['AFP_1'])
rows = np.shape(T.p)[0]
cols = np.shape(T.p)[1]
samp_names = samples['AFP_1'].to_numpy()
lengths = [len(s) for s in samp_names]
max_len = np.max(lengths)

print(f"{'Samples':^{max_len}}", end='')
[print(f'| {samp_names[i]} ', end='') for i in range(cols)]
print('')

for i in range(rows):
    print(f'{samp_names[i]:>{max_len}}', end='')
    
    for j in range(cols):
        print(f'| {p[i][j]:>{lengths[j]}n} ', end='')
        
    print('')
    
print(np.shape(T.p))
print(T.psig)

       Samples       | 3:1 HCC:HSC Sorafenib | 3:1 HCC:HSC Media | HCC Sorafenib | HCC Media | HSC Sorafenib | HSC Media 
3:1 HCC:HSC Sorafenib

NameError: name 'p' is not defined